<a href="https://colab.research.google.com/github/Hashhhhhhhh/Quantum-Optimization/blob/main/QUBO_and_QAOA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install pennylane pennylane-qiskit numpy

Refer to notes for derivations

| Item | Value $v_i$ | Weight $w_i$ |
| ---- | ----------- | ------------ |
| 0    | 10          | 5            |
| 1    | 8           | 3            |
| 2    | 6           | 2            |


Capacity, W=5

In [13]:
import numpy as np
# Knapsack parameters
values = np.array([10,8,6])
weights = np.array([5,3,2])
capacity = 5
#penalty
A = 25.0
for i in range(n):
    Q[i,i] = -values[i] + A * weights[i]**2 - 2*A*capacity*weights[i]
    for j in range(i+1, n):
        Q[i,j] = 2*A*weights[i]*weights[j]


print("QUBO matrix Q:")
print(Q)

QUBO matrix Q:
[[-635.  750.  500.]
 [   0. -533.  300.]
 [   0.    0. -406.]]


In [14]:
from pennylane.ops.qubit import PauliZ

# Coefficients for H_C
c0 = np.sum(np.diag(Q)/2) + np.sum([Q[i,j]/4 for i in range(n) for j in range(i+1,n)])
cZ = [ -Q[i,i]/2 - sum([Q[i,j]/4 for j in range(i+1,n)]) - sum([Q[j,i]/4 for j in range(i)]) for i in range(n) ]
cZZ = [ Q[i,j]/4 for i in range(n) for j in range(i+1,n) ]

print("c0:", c0)
print("cZ:", cZ)
print("cZZ:", cZZ)

c0: -399.5
cZ: [np.float64(5.0), np.float64(4.0), np.float64(3.0)]
cZZ: [np.float64(187.5), np.float64(125.0), np.float64(75.0)]


In [15]:
import pennylane as qml
dev = qml.device("default.qubit", wires=n)

In [16]:
#Defining QAOA circuit
def qaoa_layer(gamma, beta):
    # Cost layer
    for i in range(n):
        qml.RZ(2*gamma*cZ[i], wires=i)
    idx = 0
    for i in range(n):
        for j in range(i+1,n):
            qml.MultiRZ(2*gamma*cZZ[idx], wires=[i,j])
            idx += 1
    # Mixer layer
    for i in range(n):
        qml.RX(2*beta, wires=i)

@qml.qnode(dev)
def circuit(params):
    gamma, beta = params
    # Initial state: uniform superposition
    for i in range(n):
        qml.Hadamard(wires=i)
    qaoa_layer(gamma, beta)
    return qml.probs(wires=range(n))


In [17]:
#Classical optimization to find the best angles
# Cost function: expected value of QUBO
def cost_fn(params):
    probs = circuit(params)
    cost = 0
    for i, p in enumerate(probs):
        # Convert index to bitstring
        bits = [int(b) for b in format(i,"0{}b".format(n))]
        x = np.array(bits)
        cost += p * (x @ Q @ x)
    return cost

# Initial guess
params = np.array([0.1, 0.1])

opt = qml.GradientDescentOptimizer(stepsize=0.5)
for _ in range(30):
    params = opt.step(cost_fn, params)

print("Optimized gamma, beta:", params)


Optimized gamma, beta: [0.1 0.1]


In [18]:
#Measuring the solution
probs = circuit(params)
best_idx = np.argmax(probs)
best_bitstring = format(best_idx,"0{}b".format(n))
print("Best bitstring:", best_bitstring)
print("Selected items:", [i for i,b in enumerate(best_bitstring) if b=="1"])

Best bitstring: 011
Selected items: [1, 2]


This corresponds to choosing items 1 and 2, which satisfies capacity and maximizes value.